### Using clusters to replace sensor values

In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("/Users/30064457/Work/simopt/aws/data/mae-clust.csv")

column_stats = df.describe()

# Print mean +/- standard deviation of each column
print("Mean +/- Standard Deviation of each column:")
for col in df.columns:
    mean = column_stats.loc['mean', col]
    std_dev = column_stats.loc['std', col]
    print(f"{col}: {mean:.2f} +/- {std_dev:.2f}")


In [ ]:
#sensor	date	Base	Linear	Circular

# Read the CSV file
df = pd.read_csv("rob_mae_all.csv")
import pandas as pd

# Assuming df is your DataFrame
# Drop the 'sensor' column
df_without_sensor = df.drop(columns=['sensor','date'])

# Filter rows where Linear is greater than 0
filtered_df = df_without_sensor[(df_without_sensor['Linear'] > 0)&(df_without_sensor['Base'] > 0) & (df_without_sensor['Base'] < 6)]

df=filtered_df
column_stats = df.describe()

# Print mean +/- standard deviation of each column
print("Mean +/- Standard Deviation of each column:")
for col in df.columns:
    mean = column_stats.loc['mean', col]
    std_dev = column_stats.loc['std', col]
    print(f"{col}: {mean:.2f} +/- {std_dev:.2f}")


In [10]:
import pandas as pd
import sys
import numpy as np
import datetime
from io import StringIO
from typing import Optional, List
import tensorflow as tf
import math
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
from sklearn import neighbors
from sklearn.metrics import mean_squared_error 
from math import sqrt
from sklearn.preprocessing import StandardScaler
from statsmodels.tools.eval_measures import rmse


In [12]:
def load_data(bucket_name, object_key):
    df = pd.read_csv(datapath+object_key, index_col=0, parse_dates=True)
    return df

# loads data from s3 csv
def load_data_s3(bucket_name, object_key):
    client = boto3.client('s3')
    csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')

    df = pd.read_csv(StringIO(csv_string), index_col=0, parse_dates=True)
    return df

def get_training_set(df, evaluation_date: str, training_interval_hours: int, sm_station_name: str, features: List[str]):

    df_subset = df.copy().loc[df.sm_station_name == sm_station_name]
    end_datetime = np.datetime64(evaluation_date + " 00:00:00") - 1 
    training_interval = np.timedelta64(training_interval_hours, 'h')
    start_datetime = (end_datetime) - (training_interval)
    df_subset = df_subset[str(start_datetime):str(end_datetime)]
    df_subset['t'] = range(0, df_subset.shape[0])
    cols_to_keep = ['t'] + features
    df_subset = df_subset.loc[:, cols_to_keep]
    return df_subset

def get_prediction_interval(df, evaluation_date: str, sm_station_name: str, target_column):
        
    df_subset = df.copy().loc[df.sm_station_name == sm_station_name]
    start_datetime = np.datetime64(evaluation_date + " 00:00:00") 
    end_datetime = start_datetime + np.timedelta64(23, 'h')
    df_subset = df_subset[str(start_datetime):str(end_datetime)]
    df_subset['t'] = range(0, df_subset.shape[0])
    df_subset = df_subset.loc[:, ['t'] + target_column]

    return df_subset
    
def create_train_test_sets(df, n_hrs, features, target):
    train_test_sets_by_sensor = {
        "sm_station_1": {}, 
        "sm_station_2": {} 
         
        }
    sm_station_names = df.sm_station_name.unique()
    eval_dates = np.unique(df[df.is_evaluation_period == True].index.date)
    for i in range(0, sm_station_names.shape[0]):
        station_keys = list(train_test_sets_by_sensor.keys())
        train_test_sets = {}
        for date in eval_dates:
            training_interval = get_training_set(df, str(date), training_interval_hours=n_hrs, sm_station_name = sm_station_names[i], features=features)
            prediction_interval = get_prediction_interval(df, str(date), sm_station_name = sm_station_names[i], target_column=target)
            train_test_sets[f"{str(date)}"] = {"train_set": training_interval, 
                                             "test_set": prediction_interval}
    
        train_test_sets_by_sensor[station_keys[i]] = train_test_sets
    return train_test_sets_by_sensor
    
def plot_series(time, series, format="-", start=0, end=None, title=None, label=None):
    """
    Visualizes time series data

    Args:
      time (array of int) - contains the time steps
      series (array of int) - contains the measurements for each time step
      format - line style when plotting the graph
      label - tag for the line
      start - first time step to plot
      end - last time step to plot
    """

    # Setup dimensions of the graph figure
    plt.figure(figsize=(10, 6))
    
    if type(series) is tuple:

        for series_num in series:
            # Plot the time series data
            plt.plot(time[start:end], series_num[start:end], format)

    else:
      # Plot the time series data
      plt.plot(time[start:end], series[start:end], format)

    # Label the x-axis
    plt.xlabel("Time")

    # Label the y-axis
    plt.ylabel("Soil Moisture")
    
    # Label the title
    plt.title(title)
    
    plt.legend(label)

    # Overlay a grid on the graph
    plt.grid(True)

    # Draw the graph on screen
    plt.show()
    
def calculate_diff(train, test):
    # calculate change in soil moisture as sm_diff
    train['sm_diff'] = train.sm_soil_moisture.diff()
    test['sm_diff'] = test.sm_soil_moisture.diff()
    test.sm_diff.iloc[0] = test.sm_soil_moisture.iloc[0] - train.sm_soil_moisture.iloc[-1]
    return train, test


def create_train_test(train, test, features=None):
    if type(features) is tuple:
        x_train = pd.DataFrame()
        x_test = pd.DataFrame()
        for i in features:
            x_train[i] = train[i][1:]
            x_test[i] = test[i]   
    else:
        x_train = train.bom_actual_precipitation_hourly[1:]
        x_test = test.bom_actual_precipitation_hourly

    y_train = train.sm_diff[1:]
    y_test = test.sm_diff
    
    x_train = x_train.fillna(method='ffill').to_numpy()
    y_train = y_train.fillna(method='ffill').to_numpy()

    x_test = x_test.fillna(method='ffill').to_numpy()
    y_test = y_test.fillna(method='ffill').to_numpy()

    return x_train, y_train, x_test, y_test


def get_best_k(x_train, y_train): 
    mae_val = [] #to store rmse values for different k
    for K in range(20):
        K = K+1
        model = neighbors.KNeighborsRegressor(n_neighbors = K) #, weights='distance')

        model.fit(x_train[:-24], y_train[:-24])  #fit the model
        pred=model.predict(x_train[-24:]) #make prediction on test set
        error = tf.keras.metrics.mean_absolute_error(y_train[-24:], pred).numpy()
        mae_val.append(error) #store rmse values
    min_mae = min(mae_val)   
    best_k = mae_val.index(min_mae)+1

    return best_k

# calculates final predicted sm values by adding predicted diff to the last training value
def calc_final_pred(pred_diff, last_sm):
    final_pred = []
    for diff in pred_diff:
        current_sm = last_sm + diff
        last_sm = current_sm
        final_pred.append(current_sm)
    return final_pred

def run_knn(train, test, features): 
    # std_scaler= StandardScaler()
    train, test = calculate_diff(train, test)
    
    # create train and test sets using specified features
    x_train, y_train, x_test, y_test = create_train_test(train, test,features)
    # # scale values across diff features
    # x_train = std_scaler.fit_transform(x_train)
    # x_test = std_scaler.fit_transform(x_test)
    
    # get best k neighbours
    best_k = get_best_k(x_train, y_train) 

    model = neighbors.KNeighborsRegressor(n_neighbors = best_k) #, weights='distance')
    model.fit(x_train, y_train)  #fit the model
    pred_diff=model.predict(x_test) #make prediction on test set

    last_sm = train.sm_soil_moisture[-1]
    final_pred = calc_final_pred(pred_diff, last_sm)
    mae = tf.keras.metrics.mean_absolute_error(test.sm_soil_moisture, final_pred).numpy()
    mse = rmse(test.sm_soil_moisture.to_numpy(), final_pred)
    # mse = tf.keras.metrics.mean_squared_error(test.sm_soil_moisture, final_pred).numpy()
    return final_pred, mae, mse

Selects features which are loaded into train_test_sets_by_sensor. We do this to prune the overall DF into the features, dates and sensors we want to experiment with. 

In [14]:
#dfc = load_data('simpact-combined-dataset','simpact_combined_dataset.csv')
# df = df.fillna(method='ffill')

#dfc["vpd"] = (0.6108 * np.exp((17.27 * df.bom_actual_temperature)/(df.bom_actual_temperature + 237.3))) * (1.0 - (df.bom_actual_relative_humidity / 100.0))
# specify variables 

In [15]:
# load dataset 
df = load_data('simpact-evaluation-dataset','simpact_evaluation_dataset.csv')
#df = load_data('simpact_combined_for_eval','simpact_combined_for_eval.csv')
#df = df.fillna(method='ffill')

#df = load_data('simpact-combined-dataset','simpact_combined_dataset.csv')
###### from train_test_set_create...

unique_dates =  np.unique(df.index.date)
last_index =  unique_dates.shape[0] -2
mid_point = last_index / 2 
mid_point_index = int(mid_point -1)

start_date = np.unique(df.index.date)[mid_point_index]
end_date = np.unique(df.index.date)[-2] 


interval = round((end_date - start_date).days /4) # get the interval that should be between each evaluation point (we want evaluation points that are evenly spaced)
eval_date_indices = [mid_point_index, mid_point_index+(interval), mid_point_index+(interval*2), mid_point_index+(interval*3), last_index]


eval_dates = unique_dates[eval_date_indices]

# UPDATE November 20 2022: we need additional days with precipitation. We will manually select two types of conditions: one where there is a long dry period, followed by rain
# and another day of rain where there is rainfall without a dry spell beforehand 
manual_dates = [np.datetime64("2022-07-02"), np.datetime64("2022-07-22")]
eval_dates = np.append(eval_dates,manual_dates)

df['is_evaluation_period'] = False

for eval_date in eval_dates:
     df.loc[(df.index.get_level_values('datetime').date == eval_date),'is_evaluation_period'] = True
######
df["vpd"] = (0.6108 * np.exp((17.27 * df.bom_actual_temperature)/(df.bom_actual_temperature + 237.3))) * (1.0 - (df.bom_actual_relative_humidity / 100.0))


In [16]:
df.head()

,fm_station_name,trh_sensor_name,sm_station_name,sm_soil_temperature,sm_soil_moisture,sm_longitude,sm_latitude,sm_soil_moisture_interpolated,sm_soil_moisture_is_interpolated,sm_soil_temperature_interpolated,...,fm_precipitation_intensity,bom_forecasted_temperature,bom_forecasted_relative_humidity,bom_forecasted_precipitation_quantitiy,bom_actual_temperature,bom_actual_relative_humidity,bom_actual_precipitation_cumulative,bom_actual_precipitation_hourly,is_evaluation_period,vpd
datetime,,,,,,,,,,,,,,,,,,,,,
2022-05-10 00:00:00+00:00,WS400-UMB-457_1221,SENS0017-TRH-SOPA,SENS0095-SM-SOPA,NaN,NaN,151.0789,-33.84646,NaN,False,NaN,...,0.0,NaN,NaN,NaN,18.7,89.0,0.0,0.0,False,0.237226
2022-05-10 00:00:00+00:00,WS400-UMB-457_1221,SENS0017-TRH-SOPA,SENS0098-SM-SOPA,NaN,NaN,151.0787,-33.84650,NaN,False,NaN,...,0.0,NaN,NaN,NaN,18.7,89.0,0.0,0.0,False,0.237226
2022-05-10 01:00:00+00:00,WS400-UMB-457_1221,SENS0017-TRH-SOPA,SENS0095-SM-SOPA,NaN,NaN,151.0789,-33.84646,NaN,False,NaN,...,0.0,NaN,NaN,NaN,21.1,77.0,0.0,0.0,False,0.575534
2022-05-10 01:00:00+00:00,WS400-UMB-457_1221,SENS0017-TRH-SOPA,SENS0098-SM-SOPA,NaN,NaN,151.0787,-33.84650,NaN,False,NaN,...,0.0,NaN,NaN,NaN,21.1,77.0,0.0,0.0,False,0.575534
2022-05-10 02:00:00+00:00,WS400-UMB-457_1221,SENS0017-TRH-SOPA,SENS0095-SM-SOPA,NaN,NaN,151.0789,-33.84646,NaN,False,NaN,...,0.0,NaN,NaN,NaN,21.3,63.0,0.0,0.0,False,0.937286


In [17]:
# skip to dfc then replace

In [ ]:
# specify variables 
# features = ['sm_soil_moisture', 'bom_forecasted_temperature', 'bom_forecasted_relative_humidity', 'bom_forecasted_precipitation_quantitiy']
# target = ['sm_soil_moisture', 'bom_forecasted_temperature', 'bom_forecasted_relative_humidity', 'bom_forecasted_precipitation_quantitiy']
data_features = ['sm_soil_moisture', 'bom_actual_precipitation_hourly', 'bom_actual_temperature', 'bom_actual_relative_humidity', 'vpd'] # features to include in training set
target = ['sm_soil_moisture', 'bom_actual_precipitation_hourly', 'bom_actual_temperature', 'bom_actual_relative_humidity', 'vpd'] # the column with the variable we want to predict
n_hrs = 400 # the training interval in hours

# run the function 
train_test_sets_by_sensor = create_train_test_sets(df, n_hrs, data_features, target)

In [ ]:
df[df.is_evaluation_period == False].count()

In [20]:
df['sm_station_name'].unique()

array(['SENS0095-SM-SOPA', 'SENS0098-SM-SOPA', 'SENS0020-SM-SOPA',
       'SENS0107-SM-SOPA', 'SENS0152-SM-SOPA'], dtype=object)

In [21]:
df['trh_sensor_name'].unique()

array(['SENS0017-TRH-SOPA', 'SENS0007-TRH-SOPA', 'SENS0028-TRH-SOPA',
       'SENS0029-TRH-SOPA'], dtype=object)

In [22]:
print(eval_dates)

[datetime.date(2022, 7, 28) datetime.date(2022, 8, 17)
 datetime.date(2022, 9, 6) datetime.date(2022, 9, 26)
 datetime.date(2022, 10, 17) datetime.date(2022, 7, 2)
 datetime.date(2022, 7, 22)]


In [23]:
#dfc = load_data('simpact-combined-dataset','simpact_combined_dataset.csv')
dfc = load_data('simpact_combined_for_eval','simpact_combined_for_eval.csv')

In [24]:
#dfc['sm_station_name'].unique()

In [ ]:
dfc = dfc.fillna(method='ffill')

In [ ]:
# sensor_list = ['sm_station_1', 'sm_station_2', 'sm_station_3', 'sm_station_4', 'sm_station_5']
# dates = ['2022-07-02', '2022-07-22', '2022-07-28', '2022-08-17', '2022-09-06', '2022-09-26', '2022-10-17']

#sensor_list = list(train_test_sets_by_sensor.keys())
#dates = list(train_test_sets_by_sensor[sensor_list[0]].keys())


In [ ]:
#run either A here - single replace for single sensor
def replace_sensor_values(df, dfc, sm_station_name1, sm_station_name2):
    # Filter dataframes to get only relevant rows for each sensor name
    df_sensor1 = df[df['sm_station_name'] == sm_station_name1]
    dfc_sensor2 = dfc[dfc['sm_station_name'] == sm_station_name2]
    
    # Check if both sensor names exist in the dataframes
    if df_sensor1.empty or dfc_sensor2.empty:
        print("One or both sensor names not found in the dataframes.")
        return df  # Return original dataframe if either sensor name is not found
    
    # Replace values in df with corresponding values from dfc
    df.loc[df['sm_station_name'] == sm_station_name1, 'sm_soil_moisture'] = dfc_sensor2['sm_soil_moisture'].values
    
    return df

In [ ]:
#df = replace_sensor_values(df, dfc, 'SENS0098-SM-SOPA', 'SENS0099-SM-SOPA')


In [ ]:
#or B here - replace single sensor with multi sensor 
def replace_sensor_values_avg(df, dfc, sm_station_name1, sm_station_name2_list):
    # Filter dataframe to get only relevant rows for sm_station_name1
    df_sensor1 = df[df['sm_station_name'] == sm_station_name1]
    
    # Check if sm_station_name1 exists in the dataframe
    if df_sensor1.empty:
        print(f"Sensor name '{sm_station_name1}' not found in the dataframe.")
        return df  # Return original dataframe if sensor name is not found
    
    # Filter dataframe to get relevant rows for sm_station_name2_list
    dfc_sensor2 = dfc[dfc['sm_station_name'].isin(sm_station_name2_list)]
    
    # Check if any of the sm_station_name2_list sensors are not found in the dfc
    missing_sensors = set(sm_station_name2_list) - set(dfc_sensor2['sm_station_name'])
    if missing_sensors:
        print(f"Some sensor names in {missing_sensors} not found in dfc.")
        return df  # Return original dataframe if any sensor name is not found in dfc
    
    # Calculate the average of corresponding values from dfc for sm_station_name2_list
    avg_values = dfc_sensor2.groupby(dfc_sensor2.index)['sm_soil_moisture'].mean()
    
    # Replace values in df with the average values
    df.loc[df['sm_station_name'] == sm_station_name1, 'sm_soil_moisture'] = avg_values.reindex(df_sensor1.index, fill_value=0).values
    
    return df

# Example usage:
# Replace values of sensor 'SENS0020-SM-SOPA' in df with average values of sensors in the list ['SENS0107-SM-SOPA', 'SENS0152-SM-SOPA'] in dfc

# Example usage:
# Replace values of sensor 'SENS0020-SM-SOPA' in df with average values of sensors in the list ['SENS0107-SM-SOPA', 'SENS0152-SM-SOPA'] in dfc


In [ ]:
def format_sensor_names(sensor_numbers):
    formatted_sensor_names = []
    for sensor_number in sensor_numbers:
        # Pad the sensor number with leading zeros to make it four digits wide
        padded_sensor_number = str(sensor_number).zfill(4)
        # Format the sensor name
        sensor_name = f'SENS{padded_sensor_number}-SM-SOPA'
        formatted_sensor_names.append(sensor_name)
    return formatted_sensor_names

In [ ]:
# from cluster 10 of k-shape k=17
#Some sensor names in {'SENS0233-SM-SOPA'} not found in dfc.

#df = replace_sensor_values_avg(df, dfc, 'SENS0098-SM-SOPA', ['SENS0039-SM-SOPA', 'SENS0155-SM-SOPA', 'SENS0191-SM-SOPA'])
df = replace_sensor_values_avg(df, dfc, 'SENS0098-SM-SOPA', format_sensor_names([39,155,191]))

In [ ]:
# from cluster 19 of k-shape k=17
#[14,19,24,42,150,209,226,229,230]
#Some sensor names in {'SENS0150-SM-SOPA', 'SENS0226-SM-SOPA', 'SENS0014-SM-SOPA', 'SENS0229-SM-SOPA', 'SENS0230-SM-SOPA'} not found in dfc.
df = replace_sensor_values_avg(df, dfc, 'SENS0107-SM-SOPA', format_sensor_names([19,24,42,209]))

In [ ]:
# from cluster 16 of dtw k=21
#[17,19,73,118,119,134,137,185,192,203,220]

#Some sensor names in {'SENS0118-SM-SOPA', 'SENS0220-SM-SOPA', 'SENS0185-SM-SOPA'} not found in dfc.
df = replace_sensor_values_avg(df, dfc, 'SENS0098-SM-SOPA', format_sensor_names([17,19,73,119,134,137,192,203]))

In [ ]:
# from cluster 12 of dtw k=21
#[24,150,175,230,243]
#Some sensor names in {'SENS0243-SM-SOPA', 'SENS0150-SM-SOPA', 'SENS0230-SM-SOPA'} not found in dfc.
df = replace_sensor_values_avg(df, dfc, 'SENS0107-SM-SOPA', format_sensor_names([24,175]))

In [ ]:
df.head()

In [ ]:
data_features = ['sm_soil_moisture', 'bom_actual_precipitation_hourly', 'bom_actual_temperature', 'bom_actual_relative_humidity', 'vpd'] # features to include in training set
target = ['sm_soil_moisture', 'bom_actual_precipitation_hourly', 'bom_actual_temperature', 'bom_actual_relative_humidity', 'vpd'] # the column with the variable we want to predict
n_hrs = 400 # the training interval in hours

# run the function 
train_test_sets_by_sensor = create_train_test_sets(df, n_hrs, data_features, target)

In [ ]:
#sensor_list = ['sm_station_1', 'sm_station_2', 'sm_station_3', 'sm_station_4', 'sm_station_5']
#dates = ['2022-07-02', '2022-07-22', '2022-07-28', '2022-08-17', '2022-09-06', '2022-09-26', '2022-10-17']



In [ ]:
#sensor_list = list(train_test_sets_by_sensor.keys())
#dates = list(train_test_sets_by_sensor[sensor_list[0]].keys())

## KNN

In [ ]:
results = {'sensor':[], 'date':[], 'mae':[]}
# features = ('bom_forecasted_temperature', 'bom_forecasted_relative_humidity', 'bom_forecasted_precipitation_quantitiy')

train_features = ('bom_actual_precipitation_hourly', 'vpd') #'bom_actual_temperature', 'bom_actual_relative_humidity') #, 'sm_soil_moisture')
mae_list = []
mse_list = []
for sensor in sensor_list:
    # fig, ax = plt.subplots(5, sharex='col', sharey='row', figsize=(15,15))
    i = 0
    for date in dates:
        # adding break to skipp empty set
        # if sensor =='sm_station_4' and (date == '2022-10-17' or date == '2022-07-28'):
        #     break
        results['sensor'].append(sensor)
        results['date'].append(date)
        
        train = train_test_sets_by_sensor[sensor][date]['train_set']
        train = train.fillna(method='ffill')

        test = train_test_sets_by_sensor[sensor][date]['test_set']
        test = test.fillna(method='ffill')
        
        try:
            final_pred, mae, mse = run_knn(train, test, train_features)
            mae_list.append(mae)
            mse_list.append(mse)
            # results['mae'].append(mae)
            plot_series(test.t, (test.sm_soil_moisture, final_pred), title = sensor + ' ' + date, label = ('ground truth', 'forecast'))
            # plot_series(test.t, (test.bom_actual_precipitation_hourly, test.bom_actual_temperature, test.bom_actual_relative_humidity), label = features)
        except:
            pass
        


In [ ]:
# df_results = pd.DataFrame(results)
# # print(df_persistance_1_results.head())
# df_results['mae'].mean()
sum(mae_list)/len(mae_list)

In [ ]:
mae1s=pd.DataFrame(mae_list)

In [ ]:
mae1s

In [ ]:
#mae1s.iloc[14,0]=2.061128


In [ ]:
#maea=mae1s.copy()

In [ ]:
maea['ReplaceD107']=mae1s[0]

In [ ]:
#maea=maea.drop(columns=['Replace1', 'Replace2', 'ReplaceK'])

In [ ]:
#maea.rename(columns={"ReplaceK1": "ReplaceK98", "ReplaceD1": "ReplaceD98"}, inplace=True)

In [ ]:
data_to_plot = pd.concat([maea['base'], maea['ReplaceK107'], maea['ReplaceD107']], axis=1, keys=['Base', 'ReplaceK107', 'ReplaceD107'])

# Create the boxplot
sns.boxplot(data=data_to_plot)

# Add labels to the axes and a title
plt.xlabel('')
plt.ylabel('MAE Values')
plt.title('MAE Values for Base and Two Sensor Replacements')

# Show the plot
plt.show()

In [ ]:
pd.DataFrame(mae_list).describe()

In [ ]:
results_dict = {'RMSE': mse_list, 'MAE': mae_list}
pd.DataFrame(results_dict).describe()

In [ ]:
#prepare dfc - don't run as already done and csv created
df=dfc.copy()
unique_dates =  np.unique(df.index.date)
last_index =  unique_dates.shape[0] -2
mid_point = last_index / 2 
mid_point_index = int(mid_point -1)

start_date = np.unique(df.index.date)[mid_point_index]
end_date = np.unique(df.index.date)[-2] 


interval = round((end_date - start_date).days / 4) # get the interval that should be between each evaluation point (we want evaluation points that are evenly spaced)
eval_date_indices = [mid_point_index, mid_point_index+(interval), mid_point_index+(interval*2), mid_point_index+(interval*3), last_index]


eval_dates = unique_dates[eval_date_indices]

# UPDATE November 20 2022: we need additional days with precipitation. We will manually select two types of conditions: one where there is a long dry period, followed by rain
# and another day of rain where there is rainfall without a dry spell beforehand 
manual_dates = [np.datetime64("2022-07-02"), np.datetime64("2022-07-22")]
eval_dates = np.append(eval_dates,manual_dates)

df['is_evaluation_period'] = False

for eval_date in eval_dates:
     df.loc[(df.index.get_level_values('datetime').date == eval_date),'is_evaluation_period'] = True
######
df["vpd"] = (0.6108 * np.exp((17.27 * df.bom_actual_temperature)/(df.bom_actual_temperature + 237.3))) * (1.0 - (df.bom_actual_relative_humidity / 100.0))
# specify variables 

In [ ]:
df.to_csv(datapath+'simpact_combined_for_eval.csv')

In [ ]:
import folium
from IPython.display import display

def plot_sensor_locations(df_unique, sensor_list):
    # Filter dataframe to include only sensors from the provided list
    df_filtered = df_unique[df_unique['sm_station_name'].isin(sensor_list)]
    
    # Create a map with initial centering and zoom level based on all sensor locations
    map_sensor_locations = folium.Map(location=[df_filtered['sm_latitude'].mean(), df_filtered['sm_longitude'].mean()], zoom_start=10)
    
    # Add markers for each sensor location with sensor number as label
    for index, row in df_filtered.iterrows():
        folium.Marker([row['sm_latitude'], row['sm_longitude']], popup=row['sm_station_name'], 
                      tooltip=row['sensor_number']).add_to(map_sensor_locations)
    
    # Display the map directly in the Jupyter Notebook
    display(map_sensor_locations)


# Example usage:
# Assuming df_unique is your dataframe and sensor_list is a list of sensors
# Replace ['SENS0020-SM-SOPA', 'SENS0107-SM-SOPA', 'SENS0152-SM-SOPA'] with your actual list of sensors
#sensor_list = ['SENS0020-SM-SOPA', 'SENS0028-SM-SOPA', 'SENS0063-SM-SOPA', 'SENS0066-SM-SOPA']

sensor_list = ['SENS0107-SM-SOPA', 'SENS0016-SM-SOPA', 'SENS0114-SM-SOPA', 'SENS0125-SM-SOPA', 'SENS0126-SM-SOPA']
plot_sensor_locations(dfc_unique, sensor_list)


In [ ]:
sensor_list = ['SENS0098-SM-SOPA', 'SENS0099-SM-SOPA']
map_sensor_locations = plot_sensor_locations(dfc_unique, sensor_list)